# 住所から緯度経度取得
<https://qiita.com/paulxll/items/7bc4a5b0529a8d784673>

## 元ソースファイル

In [1]:
import requests
import time 
from bs4 import BeautifulSoup
from tqdm import tqdm

url = 'http://www.geocoding.jp/api/'

def coordinate(address):
    payload = {'q': address}
    html = requests.get(url, params=payload)
    soup = BeautifulSoup(html.content, "html.parser")
    if soup.find('error'):
        raise ValueError(f"Invalid address submitted. {address}")
    lat = soup.find('lat').string
    lon = soup.find('lng').string
    return lat, lon

def coordinates(addresses, interval=10, progress=True):
    coordinates = []
    for address in progress and tqdm(addresses) or addresses:
        coordinates.append(coordinate(address))
        time.sleep(interval)
    return coordinates

## 追加ソースファイル

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [21]:
def addr2geo(data_path):
    data = pd.read_csv(data_path)
    data['Latitude'] = None
    data['Longitude'] = None
    data['geometry'] = None
    for i in tqdm(range(len(data))):
        data.Latitude[i], data.Longitude[i] = coordinate(data.Address[i])
        time.sleep(10)
    data['geometry'] = gpd.points_from_xy(data.Longitude.astype(np.float64), data.Latitude.astype(np.float64))
    result = gpd.GeoDataFrame(data, geometry=data.geometry)
    return result

def geo2shp(gdf, result_path):
    gdf.to_file(driver='ESRI Shapefile', filename=result_path)